<a href="https://colab.research.google.com/github/elichen/aoc2022/blob/main/Day_22_Monkey_Map.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
input = '''        ...#
        .#..
        #...
        ....
...#.......#
........#...
..#....#....
..........#.
        ...#....
        .....#..
        .#......
        ......#.

10R5L5R10L4R5L5'''
input = input.split('\n')

In [ ]:
input = open("input.txt").readlines()
input = [x.rstrip() for x in input]

In [ ]:
import numpy as np
np.set_printoptions(edgeitems=30, linewidth=100000,
    formatter=dict(float=lambda x: "%.3g" % x))

In [ ]:
h,w = 0,0
for line in input:
  if line == "": break
  w = max(w,len(line))
  h += 1
h,w

(200, 150)

In [ ]:
map = np.zeros((h,w))
for i,line in enumerate(input):
  if line == "": break
  for j,c in enumerate(line):
    if c == ".":
      map[i,j] = 1
    if c == "#":
      map[i,j] = 2

In [ ]:
import re

In [ ]:
def find_left(row):
  for i in range(w):
    if map[row,i] != 0:
      return i
def find_right(row):
  for i in range(w-1,-1,-1):
    if map[row,i] != 0:
      return i
def find_up(col):
  for i in range(h):
    if map[i,col] != 0:
      return i
def find_down(col):
  for i in range(h-1,-1,-1):
    if map[i,col] != 0:
      return i

In [ ]:
path = input[-1]
cur_x,cur_y = find_left(0),0
facing = 0
for op in re.findall("\d+|\w", path):
  if op == 'L':
    facing = (facing-1)%4
  elif op == 'R':
    facing = (facing+1)%4
  else:
    steps = int(op)
    while steps > 0:
      if facing == 0:
        next_x, next_y = cur_x+1,cur_y
        if next_x >= w or map[next_y,next_x]==0: next_x = find_left(next_y)
      if facing == 1:
        next_x, next_y = cur_x,cur_y+1
        if next_y >= h or map[next_y,next_x]==0: next_y = find_up(next_x)
      if facing == 2:
        next_x, next_y = cur_x-1,cur_y
        if next_x < 0 or map[next_y,next_x]==0: next_x = find_right(next_y)
      if facing == 3:
        next_x, next_y = cur_x,cur_y-1
        if next_y < 0 or map[next_y,next_x]==0: next_y = find_down(next_x)
      if map[next_y, next_x] == 2:
        break
      cur_x,cur_y = next_x, next_y
      steps -= 1
  # print(op,cur_x,cur_y)
4*(cur_x+1)+1000*(cur_y+1)+facing

117054

In [ ]:
import math
from collections import deque

In [ ]:
face_size = math.gcd(w,h)

In [ ]:
faces = {
    'up': None,
    'down': None,
    'left': None,
    'right': None,
    'front': None,
    'back': None
}
face_order = {
  'up': ['right','back','left','front'],
  'down': ['right','front','left','back'],
  'left': ['back','down','front','up'],
  'right': ['front','down','back','up'],
  'front': ['left','down','right','up'],
  'back': ['right','down','left','up'],
}
def adjust_face_order(face, from_facing, from_face):
  to_facing = (from_facing+2)%4
  to_i = face_order[face].index(from_face)
  d = deque(face_order[face])
  d.rotate(to_facing-to_i)
  face_order[face] = list(d)

def traverse_face(x,y,face):
  if faces[face]: return
  faces[face] = (x//face_size,y//face_size)
  order = face_order[face]
  for facing,next_face in enumerate(order):
    if facing == 0:
      next_x = (x//face_size+1)*face_size
      if next_x < w and map[y,next_x] != 0:
        adjust_face_order(next_face,facing,face)
        traverse_face(next_x,y,next_face)
    if facing == 1:
      next_y = (y//face_size+1)*face_size
      if next_y < h and map[next_y,x] != 0:
        adjust_face_order(next_face,facing,face)
        traverse_face(x,next_y,next_face)
    if facing == 2:
      next_x = (x//face_size-1)*face_size
      if next_x >= 0 and map[y,next_x] != 0:
        adjust_face_order(next_face,facing,face)
        traverse_face(next_x,y,next_face)
    if facing == 3:
      next_y = (y//face_size-1)*face_size
      if next_y >= 0 and map[next_y,x] != 0:
        adjust_face_order(next_face,facing,face)
        traverse_face(x,next_y,next_face)
traverse_face(find_left(0),0,'up')
faces,face_order

({'up': (1, 0),
  'down': (1, 2),
  'left': (0, 2),
  'right': (2, 0),
  'front': (0, 3),
  'back': (1, 1)},
 {'up': ['right', 'back', 'left', 'front'],
  'down': ['right', 'front', 'left', 'back'],
  'left': ['down', 'front', 'up', 'back'],
  'right': ['down', 'back', 'up', 'front'],
  'front': ['down', 'right', 'up', 'left'],
  'back': ['right', 'down', 'left', 'up']})

In [ ]:
h,w

(200, 150)

In [ ]:
region_to_face = {v: k for k, v in faces.items()}
region_to_face

{(1, 0): 'up',
 (1, 2): 'down',
 (0, 2): 'left',
 (2, 0): 'right',
 (0, 3): 'front',
 (1, 1): 'back'}

In [ ]:
def find_next(x,y,dir):
  # print("find_next",x,y,dir)
  face = region_to_face[(x//face_size,y//face_size)]
  next_face = face_order[face][dir]
  next_dir = (face_order[next_face].index(face)+2)%4
  rotation = next_dir-dir
  if dir == 0: new_x, new_y = 0, y%face_size
  if dir == 1: new_x, new_y = x%face_size, 0
  if dir == 2: new_x, new_y = face_size-1, y%face_size
  if dir == 3: new_x, new_y = x%face_size, face_size-1
  if rotation == 1 or rotation == -3: new_x, new_y = face_size-1-new_y, new_x
  if rotation == 2 or rotation == -2: new_x, new_y = face_size-1-new_x, face_size-1-new_y
  if rotation == 3 or rotation == -1: new_x, new_y = new_y, face_size-1-new_x
  # print(f"old:{x,y,dir} new:{new_x+faces[next_face][0]*face_size, new_y+faces[next_face][1]*face_size, next_dir}")
  return new_x+faces[next_face][0]*face_size, new_y+faces[next_face][1]*face_size, next_dir

In [ ]:
path = input[-1]
cur_x,cur_y = find_left(0),0
facing = 0
for op in re.findall("\d+|\w", path):
  # print(op,cur_x,cur_y,facing)
  if op == 'L':
    facing = (facing-1)%4
  elif op == 'R':
    facing = (facing+1)%4
  else:
    steps = int(op)
    while steps > 0:
      if facing == 0:
        next_x,next_y,new_facing = cur_x+1,cur_y,facing
        if next_x >= w or map[next_y,next_x]==0: next_x,next_y,new_facing = find_next(cur_x,cur_y,facing)
      if facing == 1:
        next_x, next_y,new_facing = cur_x,cur_y+1,facing
        if next_y >= h or map[next_y,next_x]==0: next_x,next_y,new_facing = find_next(cur_x,cur_y,facing)
      if facing == 2:
        next_x, next_y,new_facing = cur_x-1,cur_y,facing
        if next_x < 0 or map[next_y,next_x]==0: next_x,next_y,new_facing = find_next(cur_x,cur_y,facing)
      if facing == 3:
        next_x, next_y,new_facing = cur_x,cur_y-1,facing
        if next_y < 0 or map[next_y,next_x]==0: next_x,next_y,new_facing = find_next(cur_x,cur_y,facing)
      if map[next_y, next_x] == 2:
        break
      cur_x,cur_y,facing = next_x,next_y,new_facing
      steps -= 1
      # print(steps,cur_x,cur_y,facing)
4*(cur_x+1)+1000*(cur_y+1)+facing

162096